In [1]:
import ShapeRetrieval: ShapeRetrieval as SR
using ShapeRetrieval: LearnedTimeDiffusionBlock
using SparseArrays
import ShapeRetrieval: Mesh, cot_laplacian, heat_integrator
using LinearAlgebra
using Flux

### Visualization Tools

In [33]:
bunny = SR.load_obj("./meshes/gourd.obj")
bunny = SR.normalize_mesh(bunny)
println("Area: ", sum(bunny.vertex_area))

Area: 2.681867885523121


In [34]:
function spectral_loss(model, x, λ, ϕ, A, y) 
    norm(model(x, λ, ϕ, A) - y)
end

spectral_loss (generic function with 1 method)

## Example Prediction using Spectral Mode

In [35]:
m = LearnedTimeDiffusionBlock(2, :spectral)
Flux.trainable(m::LearnedTimeDiffusionBlock) = (m.diffusion_time,)
heat_signal = zeros(bunny.nv)
heat_signal[[1, 300]] .= 1.0
λ, ϕ, A = SR.get_diffusion_inputs(bunny)
m(heat_signal, λ, ϕ, A)

0.002585682520827768

#### Sample Training 

In [36]:
data = [(heat_signal, 0.0),]
λ, ϕ, A = SR.get_diffusion_inputs(bunny)

opt_state = Flux.setup(Adam(), m);

for i=1:10000
    for d in data
        x, y  = d
        grad = gradient(spectral_loss, m, x, λ, ϕ, A, y)
        Flux.update!(opt_state, m, grad[1])
    end
end
println(m(heat_signal, λ, ϕ, A))

0.002480214795426222


In [37]:
m(heat_signal, SR.get_diffusion_inputs(bunny)...)
println(m.diffusion_time)

[2.857100931207603, 0.18106338255963728]


### AD with Implicit Mode

In [40]:
m = LearnedTimeDiffusionBlock(2, :implicit)
Flux.trainable(m::LearnedTimeDiffusionBlock) = (m.diffusion_time,)

function implicit_loss(model, x, L, M, A::Vector{Float64}, y)
    norm(model(x, L, M, A) - y)
end

inputs = SR.get_diffusion_inputs(bunny,:implicit)
m(heat_signal, inputs...)

0.3455935489505035

In [49]:
heat_signal = zeros(bunny.nv)
heat_signal[[1, 300]] .= 1.0
L, M, A = SR.get_diffusion_inputs(bunny, :implicit)
display(m(heat_signal, L, M, A))
data = [(heat_signal, bunny, 0.0),]

opt_state = Flux.setup(Adam(), m)

for i=1:1000
    for d in data
        x, mesh, y  = d
        grad = gradient(implicit_loss, m, x, L,M,A, y)
        Flux.update!(opt_state, m, grad[1])
    end
end

display(m(heat_signal, L, M, A))

0.3014815281221147

0.301481517244275